In [1]:
import glob

import numpy as np

import librosa


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler


import os
import pickle
import warnings
from sklearn import preprocessing
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 
from sklearn.metrics import f1_score
import joblib

In [2]:
# def calculate_delta(array):
   
#     rows, cols = array.shape
#     # print(rows)
#     # print(cols)
#     deltas = np.zeros((rows,20))
#     N = 2
#     for i in range(rows):
#         index = []
#         j = 1
        
#         while j <= N:
#             if i-j < 0:
#                 first =0
#             else:
#                 first = i-j
#             if i+j > rows-1:
#                 second = rows-1
#             else:
#                 second = i+j 
#             index.append((second,first))
#             j+=1
#         deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
#     return deltas
def remove_noise(data):
    data= librosa.effects.preemphasis(data)
    return data
def extract_features(data,sr):
    mfcc= librosa.feature.mfcc(y= data,sr=sr,n_mfcc=30)  
    #print(mfcc.shape) 
    delta = librosa.feature.delta(mfcc)
    #print(delta.shape)
    delta_2 = librosa.feature.delta(delta)
    #Combine MFCC and Delta features
    combined_feat = np.concatenate((mfcc,delta,delta_2))
    #features.append(combined_feat.T)
    return combined_feat.T
    #print(features)


# def extract_features(audio,rate):
       
#     mfcc_feature = mfcc.mfcc(audio, rate, 0.025, 0.01, 20, nfft = 2205, appendEnergy = True)    
#     mfcc_feature = preprocessing.scale(mfcc_feature)
# #     print(mfcc_feature)
#     delta = calculate_delta(mfcc_feature)
#     combined = np.hstack((mfcc_feature, delta)) 
#     return combined

In [3]:
feature = []
y_test = []

lstFolders = ['open', 'grant', 'unlock','others']
for lst in lstFolders:
    listFiles = glob.glob(r"./word_data/{}\*".format(lst))
    features = np.asarray(())
    finishCounter = len(listFiles)
    counter = 1
    for file in listFiles:
        try:
            audio, sr = librosa.load(file)
            vector = extract_features(audio, sr)
            
            if features.size == 0:
                features = vector
            else:
                features = np.vstack((features, vector))
                
            feature.append(vector)
            y_test.append(lstFolders.index(lst))
                
            counter += 1
            if(counter == finishCounter):
                gmm = GaussianMixture(n_components = 6, max_iter = 200, covariance_type='diag', n_init = 3)
                gmm.fit(features)
                
                # dumping the trained gaussian model
                picklefile = lst + ".joblib"
                joblib.dump(gmm, picklefile) 
        except:
            continue

In [4]:
print(y_test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [5]:
gmm_files = [ i + '.joblib' for i in ['open', 'grant', 'unlock', 'others']]

models    = [joblib.load(fname) for fname in gmm_files]
speakers   = [fname.split("\\")[-1].split(".joblib")[0] for fname 
                  in gmm_files]

In [6]:
labels = []
y = []
for i in feature:
    log_likelihood = np.zeros(len(models)) 
    for j in range(len(models)):
        gmm = models[j] 
        scores = np.array(gmm.score(i))
        print(scores)
        print("hhhhhhhhh")
        log_likelihood[j] = scores.sum()
    y.append(log_likelihood)
    #print(y)
    print("..............")
    winner = np.argmax(log_likelihood)
    print("\tdetected as - ", speakers[winner])
    labels.append(winner)
    print(labels)

-115.97045073524473
hhhhhhhhh
-125.38271725770521
hhhhhhhhh
-122.4255657781033
hhhhhhhhh
-125.2204235311632
hhhhhhhhh
..............
	detected as -  open
[0]
-116.83605261754198
hhhhhhhhh
-127.52219163095836
hhhhhhhhh
-125.00826882855218
hhhhhhhhh
-129.4365429728443
hhhhhhhhh
..............
	detected as -  open
[0, 0]
-126.09788956530001
hhhhhhhhh
-137.23367788487016
hhhhhhhhh
-134.812923810182
hhhhhhhhh
-137.08226069909443
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0]
-118.71296835947783
hhhhhhhhh
-127.57921420304346
hhhhhhhhh
-124.46954586402535
hhhhhhhhh
-126.98151764161888
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0, 0]
-120.13389590717418
hhhhhhhhh
-129.22720615103555
hhhhhhhhh
-126.39422392250039
hhhhhhhhh
-127.9553994446853
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0, 0, 0]
-118.77635483747414
hhhhhhhhh
-128.93391382376475
hhhhhhhhh
-125.63182656590148
hhhhhhhhh
-128.1609985889638
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0, 0, 0, 0]
-1

-159.64140220275132
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-122.59448624101654
hhhhhhhhh
-130.07365299213305
hhhhhhhhh
-126.301828283567
hhhhhhhhh
-127.35418146235558
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-115.22919996608272
hhhhhhhhh
-125.59075934718273
hhhhhhhhh
-122.86992735880936
hhhhhhhhh
-125.72577313683803
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-108.01801560249473
hhhhhhhhh
-113.28993663178602
hhhhhhhhh
-112.51593066244205
hhhhhhhhh
-114.75796151175517
hhhhhhhhh
..............
	detected as -  open
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-115.54375088404188
hhhhhhhhh
-125.66647365489041
hhhhhhhhh
-122.26968121119098
hhhhhh

In [7]:
f1_score(labels, y_test, average='micro')


0.9970588235294118

In [8]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(labels, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 99.71


In [9]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(labels, y_test)
print('Confusion Matrix: \n', cm)

Confusion Matrix: 
 [[100   0   0   0]
 [  0  80   0   1]
 [  0   0  80   0]
 [  0   0   0  79]]
